# **Imports and defining variables**

In [2]:
import pandas as pd
import re
import requests
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

In [3]:
navigator = 'Microsoft Edge (Scrapping; This is my system)'
url = 'https://www.auto-data.net/en'

In [4]:
html = requests.get(url, headers={'User-Agent': navigator})
html

<Response [200]>

# **Scraping**

## Creating the soup and cleaning the information to get only text

In [5]:
soup = BeautifulSoup(html.text, 'html.parser')

In [6]:
brands = soup.find_all('a', {'class' : 'marki_blok'})
brands[1]

<a class="marki_blok" href="/en/alfa-romeo-brand-11" title="Alfa Romeo - Technical Specs, Fuel consumption, Dimensions"><img alt="Alfa Romeo - Technical Specs, Fuel consumption, Dimensions" src="/img/logos/Alfa_Romeo.png"/><strong>Alfa Romeo</strong></a>

In [17]:
#This loop is to get only the part to use in link ie whatever is after 'href='
brand_list = re.findall('href="(\S+)"',str(brands))
brand_list[0:5]

['/en/acura-brand-6',
 '/en/alfa-romeo-brand-11',
 '/en/alpina-brand-16',
 '/en/aston-martin-brand-36',
 '/en/audi-brand-41']

In [16]:
#Looping to remove the /en since the url already has it and removing it makes it so it will default to another language
brand_list_final = []
for i in range(len(brand_list)):
    brand_list_final += re.findall('en/(\S+)', brand_list[i])
brand_list_final[0:5]

['acura-brand-6',
 'alfa-romeo-brand-11',
 'alpina-brand-16',
 'aston-martin-brand-36',
 'audi-brand-41']

## Creating the loops for the actual scraping

In [27]:
#Looping to get the actual url for each brand
urls_list = []
for urls in brand_list_final:
    url = 'https://www.auto-data.net/en'
    url += urls
    print(url)

url
# " ".join(urls_list)
# urls_list

https://www.auto-data.net/enacura-brand-6
https://www.auto-data.net/enalfa-romeo-brand-11
https://www.auto-data.net/enalpina-brand-16
https://www.auto-data.net/enaston-martin-brand-36
https://www.auto-data.net/enaudi-brand-41
https://www.auto-data.net/enbentley-brand-66
https://www.auto-data.net/enbmw-brand-86
https://www.auto-data.net/enbugatti-brand-106
https://www.auto-data.net/encadillac-brand-121
https://www.auto-data.net/enchevrolet-brand-156
https://www.auto-data.net/enchrysler-brand-161
https://www.auto-data.net/encitroen-brand-166
https://www.auto-data.net/encupra-brand-256
https://www.auto-data.net/endacia-brand-181
https://www.auto-data.net/endaewoo-brand-191
https://www.auto-data.net/endaihatsu-brand-2
https://www.auto-data.net/endodge-brand-32
https://www.auto-data.net/ends-brand-198
https://www.auto-data.net/enferrari-brand-62
https://www.auto-data.net/enfiat-brand-67
https://www.auto-data.net/enford-brand-72
https://www.auto-data.net/engmc-brand-97
https://www.auto-data.

'https://www.auto-data.net/enallbrands'

In [ ]:
page2 = 'https://www.auto-data.net/fr/acura-cl-model-138'         #Inside each brand, find the models

In [ ]:
page3 = 'https://www.auto-data.net/fr/acura-cl-ii-generation-7196'    #Inside each model, find the correct generation/series

In [ ]:
page4 = 'https://www.auto-data.net/fr/acura-cl-ii-3.2-i-v6-24v-type-s-263hp-automatic-31069'   #This is the final page, retrieve the desired information from here

#